In [1]:
# Данный код импортирует модуль `os`, который предоставляет функции для работы с операционной системой. 
# Этот модуль может использоваться для манипуляций с файлами и директориями, 
# для запуска команд в терминале и т.д.
import os

In [2]:
# вывод текущей рабочей директории
%pwd

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project\\research'

In [3]:
# изменяем текущую рабочую директорию на одну директорию выше относительно текущей
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Mikhnin\\Desktop\\project_1\\Text-Summarizer-Project'

In [5]:
# entity 
# В этом блоке кода определен класс DataIngestionConfig, который используется для хранения конфигурации 
# процесса загрузки данных. Класс содержит несколько полей, таких как root_dir, source_URL и local_data_file, 
# которые определяют корневую директорию, URL-адрес и локальный файл данных соответственно. 
# Также класс содержит поле unzip_dir, которое определяет директорию, в которую будут распакованы данные после загрузки.
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# configuration manager in src config
# Импортируем константы из модуля textSummarizer.constants
from textSummarizer.constants import *
# Импортируем функции read_yaml и create_directories из модуля textSummarizer.utils.common
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
# Класс ConfigurationManager определяет менеджер конфигурации, 
# который используется для загрузки конфигурационных файлов и параметров.
class ConfigurationManager:
    
    # Конструктор класса ConfigurationManager.
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        # Чтение конфигурационных файлов.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Создание директорий для хранения артефактов.
        create_directories([self.config.artifacts_root])

    # Метод get_data_ingestion_config возвращает конфигурацию процесса загрузки данных.
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        # Получение конфигурации процесса загрузки данных.
        config = self.config.data_ingestion

        # Создание директорий для хранения данных.
        create_directories([config.root_dir])

        # Создание объекта DataIngestionConfig, который содержит конфигурацию процесса загрузки данных.
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        # Возвращение объекта DataIngestionConfig.
        return data_ingestion_config

In [8]:
# created conponent

import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
# Определяем класс DataIngestion, который используется для загрузки и распаковки данных.
class DataIngestion:
    
    # Конструктор класса DataIngestion.
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Метод download_file загружает файл данных из URL-адреса и сохраняет его в локальной директории.
    def download_file(self):
        
        # Проверяем, существует ли локальный файл данных.
        if not os.path.exists(self.config.local_data_file):
            
            # Если локальный файл данных не существует, то загружаем его из URL-адреса.
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            
            # Выводим сообщение о том, что файл данных был успешно загружен.
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        # Если локальный файл данных уже существует, то выводим сообщение о его размере.
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    # Метод extract_zip_file распаковывает архив с данными в локальной директории.
    def extract_zip_file(self):
        
        # Определяем директорию, в которую будут распакованы данные.
        unzip_path = self.config.unzip_dir
        
        # Создаем директорию, если она не существует.
        os.makedirs(unzip_path, exist_ok=True)        

        # Распаковываем архив с данными в директорию unzip_path.
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# created the pipeline
try:
    
# Создаем объект ConfigurationManager, который загружает конфигурационные файлы и параметры.
    config = ConfigurationManager()

# Получаем конфигурацию процесса загрузки данных.
    data_ingestion_config = config.get_data_ingestion_config()

# Создаем объект DataIngestion, который используется для загрузки и распаковки данных.
    data_ingestion = DataIngestion(config=data_ingestion_config)

# Загружаем файл данных из URL-адреса и сохраняем его в локальной директории.
    data_ingestion.download_file()

# Распаковываем архив с данными в локальной директории.
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2023-08-19 01:35:00,623: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-19 01:35:00,629: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-19 01:35:00,632: INFO: common: created directory at: artifacts]
[2023-08-19 01:35:00,634: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-19 01:35:03,269: INFO: 2875172352: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DDD0:3F51:1FA1426:20D909C:64DFF214
Accept-Ranges: bytes
Date: Fri, 18 Aug 2023 22:35:01 GMT
Via: 1.1 varnish
X-Served-By: cache-hel1410024-HEL
X-Cache: